# Converter from pommesinvest to AMIRIS
Convert pommesinvest results to AMIRIS input data format

## Package imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pommesevaluation.amiris_converter import (
    convert_annual_data_to_fame_time, convert_time_series_index_to_fame_time
)

## Notebook and workflow settings

In [ ]:
# Paths to data needed to be converted
path_model_inputs = "./model_inputs/"
path_processed_outputs = "./data_out/"

# Filename and scenarios
filename_investment = "results_total_dr_scenario"
filename_backup = "installed_capacity_backup_generation.csv"
filename_storage = "installed_storage_capacity.csv"
file_name_res_capacities = "sources_renewables_investment_model.csv"
file_name_res_generation = "sources_renewables_ts_hourly.csv"
file_name_demand = ""

dr_scenarios = ["none", "5", "50", "95"]

# Define which pieces of information to exclude from investments
exclude_from_investment = [
    "PHS_inflow", 
    "battery_inflow", 
    "hoho_cluster_shift_only",
    "hoho_cluster_shift_shed",
    "hydrogen_electrolyzer",
    "ind_cluster_shed_only",
    "ind_cluster_shift_only",
    "ind_cluster_shift_shed",
    "tcs+hoho_cluster_shift_only",
    "tcs_cluster_shift_only",
]

demand_response_focus_cluster = "ind_cluster_shift_only"
exclude_from_investment.remove(demand_response_focus_cluster)

# Read in and convert
## Transformers & renewables
* Investment results (transformers only)
* Backup generation (exogenous transformers)
* Renewable capacities & generation

In [ ]:
invest_results = pd.read_csv(f"{path_processed_outputs}{filename_investment}_{dr_scenarios[2]}.csv", index_col=0)
invest_results = invest_results.loc[[idx for idx in invest_results.index if idx not in exclude_from_investment]]
backup_generation = pd.read_csv(f"{path_processed_outputs}{filename_backup}", index_col=0)

res_capacities_2020 = pd.read_csv(f"{path_model_inputs}{file_name_res_capacities}", index_col=0)
res_capacities_2020 = res_capacities_2020.loc[res_capacities_2020["country"] == "DE"]
res_generation = pd.read_csv(f"{path_model_inputs}{file_name_res_generation}", index_col=0)

In [ ]:
_ = convert_annual_data_to_fame_time(
    invest_results, 
    save=True, 
    path="./data_out/amiris/",
    filename="installed_capacity_ts"
)
_ = convert_annual_data_to_fame_time(
    backup_generation,
    orient="rows",
    save=True,
    path="./data_out/amiris/",
    filename="exogenous_installed_capacity_ts"
)
_ = convert_time_series_index_to_fame_time(
    res_generation,
    save=True,
    path="./data_out/amiris/",
    filename="res_generation_ts"
)

## Net Demand
Since in AMIRIS, there is a shortcoming in terms of modelling competing demand-side flexibility options, all flexibility options except the one demand response-cluster that is focused upon are not modelled explicitly, but their dispatch is considered in calculating a net demand, i.e. a demand after
* imports and exports,
* storages,
* demand response (except for the focus cluster)
* electrolyzer operation